In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [5]:
# OneHot encoding for Geography column
from sklearn.preprocessing import OneHotEncoder
one_hot_encoding_geography = OneHotEncoder()
geography_encoded = one_hot_encoding_geography.fit_transform(data[['Geography']])
data_onehot_geography = pd.DataFrame(geography_encoded.toarray(),columns=one_hot_encoding_geography.get_feature_names_out(['Geography']))
data = pd.concat([data,data_onehot_geography],axis=1)
data = data.drop(['Geography'],axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [6]:
# split the data into features and target variable
X = data.drop(['EstimatedSalary'], axis=1)
y = data['EstimatedSalary']


In [7]:
# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# save all the files
with open('Reg_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('Reg_label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('Reg_one_hot_encoding_geography.pkl', 'wb') as f:
    pickle.dump(one_hot_encoding_geography, f)

# ANN Regression

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


Matplotlib is building the font cache; this may take a moment.


In [11]:
# Build the model
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1) # Output layer for regression (no activation function)
    ]
)

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Setup the tensorboard
import datetime
log_dir = "regression/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
# Setup EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [16]:
# Train the model

history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 13502259200.0000 - mae: 101043.5078 - val_loss: 13009620992.0000 - val_mae: 98531.0938
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 13330765824.0000 - mae: 100485.8203 - val_loss: 12711979008.0000 - val_mae: 97026.1016
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 12798607360.0000 - mae: 97681.4141 - val_loss: 11928705024.0000 - val_mae: 93062.5859
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 12104412160.0000 - mae: 93838.8828 - val_loss: 10614097920.0000 - val_mae: 86421.7188
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 10405245952.0000 - mae: 85543.6953 - val_loss: 8901270528.0000 - val_mae: 77840.6797
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 8500418560.0000 - mae: 76031.4062 - val_loss: 7092433408.0000 - val_mae: 68824.5312
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6725090304.0000 - mae: 66935.0938 - val_loss: 5532272

In [17]:
# Load tensorboard extension
%load_ext tensorboard

In [19]:
%tensorboard --logdir regression/fit

Reusing TensorBoard on port 6006 (pid 8560), started 0:03:32 ago. (Use '!kill 8560' to kill it.)

In [20]:
model.save("regression_model.h5")